In [4]:
import pandas as pd
import torch
import re


# GPU 사용 가능 -> True, GPU 사용 불가 -> False
print(torch.cuda.is_available())

True


In [5]:
train_df = pd.read_csv('./data/train.csv', encoding='utf-8')
test_df = pd.read_csv('./data/test.csv', encoding='utf-8')

In [6]:
train_df.head()

,ID,input,output
0,TRAIN_00000,별 한 게토 았깝땀. 왜 싸람듯릭 펼 1캐를 쥰눈징 컥꺾폰 싸람믐롯섞 맒록 섧멍핥쟈...,별 한 개도 아깝다. 왜 사람들이 별 1개를 주는지 겪어본 사람으로서 말로 설명하자...
1,TRAIN_00001,잚많 쟉꼬 갉 태 좋눼욥. 차못동 줆 ㅋ,잠만 자고 갈 때 좋네요. 잠옷도 줌 ㅋ
2,TRAIN_00002,절테 간면 않 된는 굣 멥몫,절대 가면 안 되는 곳 메모
3,TRAIN_00003,야... 칵컥 좋꾜 부됴 뼝 뚫렷썹 신원햐쥠만 닮패 넴센 밌쪄벅림. 샥퀘 핥류만 묵...,아... 가격 좋고 뷰도 뻥 뚫려서 시원하지만 담배 냄새 미쳐버림. 싸게 하루만 묵...
4,TRAIN_00004,집윈 축쳐눌료 딴너왓눈뎁 카셩뷔 좋곱 칼쿰한네올. 쩌럼한뒈 뮬콰 욺료토 잊쿄 빻토 ...,지인 추천으로 다녀왔는데 가성비 좋고 깔끔하네요. 저렴한데 물과 음료도 있고 방도 ...


In [7]:
# 단어 분리
match_words = {}
for index, (input, output) in train_df.iloc[:, 1:].iterrows():
    input_words = input.split()
    output_words = output.split()
    
    for iw, ow in zip(input_words, output_words):
        match_words[iw] = ow
        
len(match_words)

173020

In [8]:
# 특수 문자 제외하고 숫자, 또는 영문자 하나만 있는 경우 제외
def is_valid(text):
    # 영어, 숫자, 특수문자 제거
    cleaned_text = re.sub(r"[a-zA-Z0-9]", "", text)  # 영어와 숫자 제거
    cleaned_text = re.sub(r"[^가-힣]", "", cleaned_text)  # 한글을 제외한 나머지 문자 제거
    
    # 비어있는 경우
    if cleaned_text == '':
        return True

    return False  # 그 외의 경우(한글)는 False


match_words = {key: value for key, value in match_words.items() if not is_valid(key)}
len(match_words)

172320

In [17]:
df = pd.DataFrame({'obfuscation': match_words.keys(), 'original': match_words.values()})

voca = {}
for index, (of, ori) in df.iterrows():
    voca[of+ori] = index
    
df.head()

,obfuscation,original
0,별,별
1,한,한
2,게토,개도
3,았깝땀.,아깝다.
4,왜,외


In [20]:
df['voca_label'] = df.apply(lambda row: voca[row['obfuscation'] + row['original']], axis=1)
df.head()

,obfuscation,original,voca_label
0,별,별,0
1,한,한,1
2,게토,개도,2
3,았깝땀.,아깝다.,3
4,왜,외,4
